In [9]:
#iris plant classification data set built from scikit_learn library
from sklearn.datasets import load_iris 
dataset = load_iris()

X = dataset.data  #Eigenvalues
Y = dataset.target #category


attribute_mean = X.mean(axis=0)
X_d = np.array(X >= attribute_mean, dtype='int') # transfer continuous value to discrete discretization of continuous values


In [10]:
X_d

array([[0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [15]:
model

{'feature': 2, 'predictor': {0: 0, 1: 2}}

In [14]:
sorted(errors.items(), key=itemgetter(1))

[(2, 50), (3, 50), (0, 56), (1, 66)]

In [1]:
import numpy as np
from collections import defaultdict
from operator import itemgetter

def train_feature_value(X, y_true, feature_index, value):
    # create a dictionary to count how frequenctly a sample given a specific feature appears in certian class
    #Create a dictionary to count the frequency of occurrence of a feature in a category
    class_counts = defaultdict(int)
    for sample, y in zip(X,y_true):
        if sample[feature_index] == value:
            class_counts[y] += 1
            
    # get the best one by sorting The category in which the feature value is most likely to belong
    sorted_class_counts = sorted(class_counts.items(), key=itemgetter(1), reverse=True)
    most_frequent_class = sorted_class_counts[0][0]
    
    #error is the number of samples that do not classified as the most frequent class 1- eigenvalues belonging to most_frequent_class
    incorrect_predictions = [class_count for class_value, class_count in class_counts.items() if class_value != most_frequent_class]
    error = sum(incorrect_predictions)
    return error
   
    
def train_on_value(X, y_true, feature_index):
    print(feature_index)
    values = set(X[:, feature_index])
    #errors = [None] * len(values)
    errors = []
    #pool = mp.Pool(mp.cpu_count()-1)
    #errors = [pool.apply_async(train_feature_value, (X, y_true, feature_index, v)).get() for v in values]
    
    
    for v in values:
        error = train_feature_value(X, y_true, feature_index, v)
        errors.append(error)
        
    
    total_error = sum(errors)
    return total_error

In [69]:
import multiprocessing as mp

def oneR_feature_selection(X, y_true):
    
    errors = {}
    
    for feature_index in range(X.shape[1]):
        print(feature_index)
        
        error = train_on_value(X, y_true, feature_index)
        errors[feature_index] = error

    feature_ranking, errors = sorted(errors.items(), key=itemgetter(1))
    return feature_ranking

In [24]:
import multiprocessing as mp

def train_on_value_wrapper(args):
    print(args)
    return train_on_value(*args)


def oneR_feature_selection(X, y_true):
    
    
    pool = mp.Pool(mp.cpu_count()-1)
    args_generator = ((f_index, [X, y_true, f_index]) for f_index in range(X.shape[1]))
    errors = dict(zip(range(X.shape[1]), pool.map(train_on_value_wrapper, args_generator,)))
    """
    errors = dict(
        (f_index, 
            pool.apply_async(train_on_value, (X, y_true, f_index, )).get()) 
            for f_index in range(X.shape[1]))
    """
    feature_ranking, errors = sorted(errors.items(), key=itemgetter(1))
    return feature_ranking

In [3]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

In [4]:
dataset_path = "/home/colombelli/Documents/research_proc_ds/BRCA/processedBRCA.rds"
read_RDS = robjects.r['readRDS']
df = read_RDS(dataset_path)

with localconverter(robjects.default_converter + pandas2ri.converter):
    pydf = robjects.conversion.rpy2py(df)

pydf

,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,tAKR,class
TCGA.A7.A0CE.11A.21R.A089.07,1.835256,3.554033,1.725914,0.053712,2.609986,10.066793,0.827885,0.239493,5.302355,0.192659,...,1.995186,3.262671,3.369012,1.563142,3.604153,8.470858,3.930036,4.813726,0.072789,0.0
TCGA.A7.A0CH.11A.32R.A089.07,1.465276,3.044173,1.336881,0.000000,2.510951,9.678872,0.836471,0.902070,5.751827,0.063942,...,1.988714,2.998502,3.180680,1.101841,3.652771,8.285092,4.003378,4.142621,0.000000,0.0
TCGA.A7.A0D9.11A.53R.A089.07,0.771441,1.265881,0.397957,0.016972,2.664702,10.236018,0.896903,1.659686,5.106264,1.373948,...,2.367464,2.999131,3.380447,0.074236,4.557467,7.111538,4.505654,4.440348,0.057081,0.0
TCGA.A7.A0DB.11A.33R.A089.07,1.250850,1.432627,0.420242,0.000000,3.056454,10.213127,0.681999,1.510526,5.486720,0.592994,...,2.381241,3.126666,3.446197,0.116791,4.558598,7.985802,4.409248,4.329472,0.174911,0.0
TCGA.A7.A0DC.11A.41R.A089.07,1.780794,2.472766,1.217222,0.018459,2.867500,9.835526,0.485838,0.978640,4.993324,0.230467,...,2.735298,4.341991,3.696360,0.804860,4.708484,7.856840,4.675049,5.062417,0.121459,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01A.11R.A41B.07,2.950158,4.764773,1.405402,0.049774,1.522249,6.967726,0.228381,0.188093,4.312322,0.066562,...,1.714493,2.476397,2.843559,1.888288,2.364388,9.004533,3.097405,3.599643,0.163136,1.0
TCGA.XX.A899.01A.11R.A36F.07,2.078279,4.165493,1.781945,0.024371,3.095815,9.323539,0.151847,0.744892,5.183241,0.166356,...,2.492434,3.964144,3.869465,0.935256,3.742199,8.384014,4.665110,4.574820,0.158603,1.0
TCGA.XX.A89A.01A.11R.A36F.07,1.627900,4.239813,2.109432,0.214820,2.331652,8.953693,1.556855,0.477132,5.343588,0.517982,...,2.039000,3.022114,3.848598,2.774375,3.712929,8.347144,4.130080,4.929970,0.224645,1.0
TCGA.Z7.A8R5.01A.42R.A41B.07,1.764846,4.724472,1.238668,0.000000,1.913517,9.569095,0.085615,1.000764,4.500141,0.000000,...,1.789647,2.307986,2.721709,1.180616,2.796347,8.122177,2.897322,4.158953,0.000000,1.0


In [5]:
import numpy as np

data = np.array(pydf.iloc[0:, 0:len(pydf.columns)-1])
y = np.array(pydf.iloc[0:, len(pydf.columns)-1:]).flatten()

In [25]:
oneR_feature_selection(data, y)

(0, [array([[1.83525621, 3.55403265, 1.72591369, ..., 3.93003581, 4.8137258 ,
        0.07278856],
       [1.46527598, 3.04417267, 1.33688082, ..., 4.00337758, 4.14262148,
        0.        ],
       [0.77144086, 1.26588092, 0.3979566 , ..., 4.50565403, 4.44034846,
        0.05708108],
       ...,
       [1.6279004 , 4.23981256, 2.10943219, ..., 4.13007958, 4.92996994,
        0.22464511],
       [1.76484637, 4.72447226, 1.23866751, ..., 2.89732249, 4.15895282,
        0.        ],
       [1.5502037 , 4.2955547 , 1.6999011 , ..., 2.93153706, 3.92800286,
        0.04468243]]), array([0., 0., 0., ..., 1., 1., 1.]), 0])
(835, [array([[1.83525621, 3.55403265, 1.72591369, ..., 3.93003581, 4.8137258 ,
        0.07278856],
       [1.46527598, 3.04417267, 1.33688082, ..., 4.00337758, 4.14262148,
        0.        ],
       [0.77144086, 1.26588092, 0.3979566 , ..., 4.50565403, 4.44034846,
        0.05708108],
       ...,
       [1.6279004 , 4.23981256, 2.10943219, ..., 4.13007958, 4.92996994,
 

        0.04468243]]), array([0., 0., 0., ..., 1., 1., 1.]), 10020])
(10855, [array([[1.83525621, 3.55403265, 1.72591369, ..., 3.93003581, 4.8137258 ,
        0.07278856],
       [1.46527598, 3.04417267, 1.33688082, ..., 4.00337758, 4.14262148,
        0.        ],
       [0.77144086, 1.26588092, 0.3979566 , ..., 4.50565403, 4.44034846,
        0.05708108],
       ...,
       [1.6279004 , 4.23981256, 2.10943219, ..., 4.13007958, 4.92996994,
        0.22464511],
       [1.76484637, 4.72447226, 1.23866751, ..., 2.89732249, 4.15895282,
        0.        ],
       [1.5502037 , 4.2955547 , 1.6999011 , ..., 2.93153706, 3.92800286,
        0.04468243]]), array([0., 0., 0., ..., 1., 1., 1.]), 10855])
(11690, [array([[1.83525621, 3.55403265, 1.72591369, ..., 3.93003581, 4.8137258 ,
        0.07278856],
       [1.46527598, 3.04417267, 1.33688082, ..., 4.00337758, 4.14262148,
        0.        ],
       [0.77144086, 1.26588092, 0.3979566 , ..., 4.50565403, 4.44034846,
        0.05708108],
       

        0.04468243]]), array([0., 0., 0., ..., 1., 1., 1.]), 20875])
(21710, [array([[1.83525621, 3.55403265, 1.72591369, ..., 3.93003581, 4.8137258 ,
        0.07278856],
       [1.46527598, 3.04417267, 1.33688082, ..., 4.00337758, 4.14262148,
        0.        ],
       [0.77144086, 1.26588092, 0.3979566 , ..., 4.50565403, 4.44034846,
        0.05708108],
       ...,
       [1.6279004 , 4.23981256, 2.10943219, ..., 4.13007958, 4.92996994,
        0.22464511],
       [1.76484637, 4.72447226, 1.23866751, ..., 2.89732249, 4.15895282,
        0.        ],
       [1.5502037 , 4.2955547 , 1.6999011 , ..., 2.93153706, 3.92800286,
        0.04468243]]), array([0., 0., 0., ..., 1., 1., 1.]), 21710])
(22545, [array([[1.83525621, 3.55403265, 1.72591369, ..., 3.93003581, 4.8137258 ,
        0.07278856],
       [1.46527598, 3.04417267, 1.33688082, ..., 4.00337758, 4.14262148,
        0.        ],
       [0.77144086, 1.26588092, 0.3979566 , ..., 4.50565403, 4.44034846,
        0.05708108],
       

TypeError: train_on_value() missing 1 required positional argument: 'feature_index'

In [12]:
aa = ["a", 1, 3]

def foo(a, um, tres):
    print(a, um, tres)

def foo_w(args):
    return foo(*args)

foo_w(aa)

a 1 3
